# Dataset Overview

In [ ]:
import os 
import time
import matplotlib.pyplot as plt
from datetime import datetime
import pandas as pd
import numpy as np
import time
from datetime import datetime
"need to do pip install plotly"
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 14})

from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf

import warnings # Supress warnings
warnings.filterwarnings("ignore")


# Data processing

### 1. retreive Bitcoin pre-process it(time_stamp to human readable) 
### 2. visualize data using candle stick



In [ ]:
import pandas as pd
import numpy as np

from datetime import datetime

import plotly.graph_objects as go
from plotly.subplots import make_subplots

import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 14})

from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf

import warnings # Supress warnings
warnings.filterwarnings("ignore")

data_folder = "/content"

asset_details = pd.read_csv('/content/asset_details.csv')
data = pd.read_csv("/content/train.csv")

dic = {}
for a in asset_details['Asset_ID']:
    dic[a] = asset_details[asset_details.Asset_ID == a].Asset_Name.values[0]

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


FileNotFoundError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# data size is 24 miliion
data.shape

In [ ]:
# Asset_ID correspond to Asset_Name
asset_details

In [ ]:
data[:5]


In [ ]:
Bitcoin = data[data['Asset_ID'] ==1]
Bitcoin[:5]

# Data features

timestamp - Unix timestamps (the number of seconds elapsed since 1970-01-01 00:00:00.000 UTC). Timestamps in this dataset are multiple of 60, indicating minute-by-minute data.

Asset_ID - uniquely identifies the traded coin

Count - number of trades executed within the respective minute

Open, High, Low, Close - the usual price details for a given unit of time.

Volume - amount of units of this coin traded in the particular minute

VWAP - The average price of the asset over the time interval, weighted by volume. VWAP is an aggregated form of trade data.

Target - Residual log-returns for the asset over a 15 minute horizon <- we know this from the competition's official description.

# Pre-Processing
# Convert timestamp
### resample the minute-wise crypto data to daily samples. This reduces the amount of samples from 24,236,806 to 1,360.

In [ ]:
# Convert timestamp to single day(from one minute)
data['timestamp'] = data['timestamp'].astype('datetime64[s]')

# Resample
daily_data = pd.DataFrame()

for asset_id in asset_details.Asset_ID:
    single_daily_data = data[data.Asset_ID == asset_id].copy()
    single_daily_data_new = single_daily_data[['timestamp','Count']].resample('D', on='timestamp').sum()
    single_daily_data_new['Open'] = single_daily_data[['timestamp','Open']].resample('D', on='timestamp').first()['Open']
    single_daily_data_new['High'] = single_daily_data[['timestamp','High']].resample('D', on='timestamp').max()['High']
    single_daily_data_new['Low'] = single_daily_data[['timestamp','Low']].resample('D', on='timestamp').min()['Low']
    single_daily_data_new['Close'] = single_daily_data[['timestamp','Close']].resample('D', on='timestamp').last()['Close']
    single_daily_data_new['Volume'] = single_daily_data[['timestamp','Volume']].resample('D', on='timestamp').sum()['Volume']
    single_daily_data_new['Asset_ID'] = asset_id

    daily_data = daily_data.append(single_daily_data_new.reset_index(drop=False))
    
daily_data = daily_data.sort_values(by = ['timestamp', 'Asset_ID']).reset_index(drop=True)
daily_data = daily_data.pivot(index='timestamp', columns='Asset_ID')[['Count', 'Open', 'High', 'Low', 'Close', 'Volume']]
daily_data = daily_data.reset_index(drop=False)

display(daily_data.head(10))

In [ ]:
daily_data.Volume

# Visualize the dataset price

In [ ]:
# Visualize
figure = make_subplots(
    rows=len(asset_details.Asset_ID), cols=1,
    subplot_titles=(asset_details.Asset_Name)
)

for i, asset_id in enumerate(asset_details.Asset_ID):
    figure.append_trace(go.Candlestick(x=daily_data.timestamp, 
                                         open=daily_data[('Open', asset_id)], 
                                         high=daily_data[('High', asset_id)], 
                                         low=daily_data[('Low', asset_id)], 
                                         close=daily_data[('Close', asset_id)]),
                  row=i+1, col=1,
                    )

    figure.update_xaxes(range=[daily_data.timestamp.iloc[0], daily_data.timestamp.iloc[-1]], row=i+1, col=1)


figure.update_layout(xaxis_rangeslider_visible = False, 
                  xaxis2_rangeslider_visible = False, 
                  xaxis3_rangeslider_visible = False,
                  xaxis4_rangeslider_visible = False,
                  xaxis5_rangeslider_visible = False,
                  xaxis6_rangeslider_visible = False,
                  xaxis7_rangeslider_visible = False,
                  xaxis8_rangeslider_visible = False,
                  xaxis9_rangeslider_visible = False,
                  xaxis10_rangeslider_visible = False,
                  xaxis11_rangeslider_visible = False,
                  xaxis12_rangeslider_visible = False,
                  xaxis13_rangeslider_visible = False,
                  xaxis14_rangeslider_visible = False,
                  height=3000, width=800, 
                  #title_text="Subplots with Annotations"
                      margin = dict(
        l = 0,
        r = 0,
        b = 0,
        t = 30,
        pad = 0)
                 )
                 
figure.show()

# Draw the candlestick diagram (you can click on "autoscale in the diagram")

In [ ]:
# Visualize
figure = make_subplots(
    rows=2, cols=1,
    subplot_titles=(['Bitcoin', 'Ethereum'])
)

for i, asset_id in enumerate([1, 6]):
    figure.append_trace(go.Candlestick(x=daily_data.timestamp, 
                                         open=daily_data[('Open', asset_id)], 
                                         high=daily_data[('High', asset_id)], 
                                         low=daily_data[('Low', asset_id)], 
                                         close=daily_data[('Close', asset_id)]),
                  row=i+1, col=1,
                    )

    figure.update_xaxes(range=[pd.Timestamp('2018-01-01'), pd.Timestamp('2018-03-01')], row=i+1, col=1)

figure.update_yaxes(range=[0, 20000], row=1, col=1)
figure.update_yaxes(range=[0, 2000], row=2, col=1)

figure.update_layout(xaxis_rangeslider_visible = False, 
                  xaxis2_rangeslider_visible = False, 
                  #height=3000, 
                  width=800, 
                  #title_text="Subplots with Annotations"
                      margin = dict(
        l = 0,
        r = 0,
        b = 0,
        t = 30,
        pad = 0)
                 )
                 
figure.show()

In [ ]:
daily_data['year'] = pd.DatetimeIndex(daily_data['timestamp']).year
daily_data['quarter'] = pd.DatetimeIndex(daily_data['timestamp']).quarter
daily_data['month'] = pd.DatetimeIndex(daily_data['timestamp']).month
daily_data['weekofyear'] = pd.DatetimeIndex(daily_data['timestamp']).weekofyear
daily_data['dayofyear'] = pd.DatetimeIndex(daily_data['timestamp']).dayofyear
daily_data['weekday'] = pd.DatetimeIndex(daily_data['timestamp']).weekday
daily_data.shape

## Show volume Histograms

In [ ]:
#histogram different representation of volume for ALL coins
daily_data_copy = daily_data.Volume

daily_data_copy.hist(figsize=(20,10))

plt.title('Evolution of Volume Bitcoin')
plt.show()

## Volume visualization for bitcoin, ethereum, dogecoin for all three years

In [ ]:
#f, (ax1,ax2,ax3) = plt.subplots(nrows=1, ncols=1, figsize=(15,19))
f, ax1 = plt.subplots(nrows=1, ncols=1, figsize=(15,8))
#bitcoin
sns.lineplot(data=train_daily, x='timestamp', y = ('Volume',  1) , ax=ax1, color='blue');
ax1.set_title('Bitcoin Volume')
ax1.set_xlim([train_daily.timestamp.iloc[0], train_daily.timestamp.iloc[-1]])

#ethereum
# sns.lineplot(x=train_daily['timestamp'], y = trend_vol, ax=ax, color='red');
# sns.lineplot(data=train_daily, x='timestamp', y = ('Volume', 6) , ax=ax1, color='blue');
# ax1.set_title('Ethereum Volume')
# ax1.set_xlim([train_daily.timestamp.iloc[0], train_daily.timestamp.iloc[-1]])

#dogecoin
# sns.lineplot(data=train_daily, x='timestamp', y = ('Volume', 4) , ax=ax1, color='blue');
# ax1.set_title('Dogecoin Volume')
# ax1.set_xlim([train_daily.timestamp.iloc[0], train_daily.timestamp.iloc[-1]])

#plot and save figure
plt.show()
f.savefig('Volume_bitcoin.png', transparent = True)

NameError: ignored

Price trend of different coins

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

periods = [3, 3, 12]
   
asset_id = 1 # Bitcoin
# Visualize
f, ax = plt.subplots(nrows=len(periods), ncols=1, figsize=(12, 8))
i=0
    
asset_id=1
decomp = seasonal_decompose(train_daily[('Close',  asset_id)].fillna(0), period=28, model='additive', extrapolate_trend='freq')
train_daily[(f'Trend_{28}',  asset_id)] = np.where(train_daily[('Close',  asset_id)].isna(), np.NaN, decomp.trend) #decomp.trend


sns.lineplot(data=train_daily, x='timestamp', y = ('Close',  asset_id) , ax=ax[i], color='lightgrey');
sns.lineplot(data=train_daily, x='timestamp', y = (f'Trend_{28}',  asset_id) , ax=ax[i], color='red');
ax[i].set_title(f"{asset_details[asset_details.Asset_ID == asset_id].Asset_Name.values[0]} Trend ")
ax[i].set_xlim([train_daily.timestamp.iloc[0], train_daily.timestamp.iloc[-1]])
#ax[i].set_ylim([-0.6,0.6])
ax[i].set_ylabel('Close Price [$]')  
i=1
asset_id=6
decomp = seasonal_decompose(train_daily[('Close',  asset_id)].fillna(0), period=28, model='additive', extrapolate_trend='freq')
train_daily[(f'Trend_{28}',  asset_id)] = np.where(train_daily[('Close',  asset_id)].isna(), np.NaN, decomp.trend) #decomp.trend


sns.lineplot(data=train_daily, x='timestamp', y = ('Close',  asset_id) , ax=ax[i], color='lightgrey');
sns.lineplot(data=train_daily, x='timestamp', y = (f'Trend_{28}',  asset_id) , ax=ax[i], color='red');
ax[i].set_title(f"{asset_details[asset_details.Asset_ID == asset_id].Asset_Name.values[0]} Trend ")
ax[i].set_xlim([train_daily.timestamp.iloc[0], train_daily.timestamp.iloc[-1]])
#ax[i].set_ylim([-0.6,0.6])
ax[i].set_ylabel('Close Price [$]')
i=2
asset_id=4
decomp = seasonal_decompose(train_daily[('Close',  asset_id)].fillna(0), period=28, model='additive', extrapolate_trend='freq')
train_daily[(f'Trend_{28}',  asset_id)] = np.where(train_daily[('Close',  asset_id)].isna(), np.NaN, decomp.trend) #decomp.trend


sns.lineplot(data=train_daily, x='timestamp', y = ('Close',  asset_id) , ax=ax[i], color='lightgrey');
sns.lineplot(data=train_daily, x='timestamp', y = (f'Trend_{28}',  asset_id) , ax=ax[i], color='red');
ax[i].set_title(f"{asset_details[asset_details.Asset_ID == asset_id].Asset_Name.values[0]} Trend ")
ax[i].set_xlim([train_daily.timestamp.iloc[0], train_daily.timestamp.iloc[-1]])
#ax[i].set_ylim([-0.6,0.6])
ax[i].set_ylabel('Close Price [$]')
#plt.suptitle(f'Underlying Trend with {PERIOD} day period\n')
plt.tight_layout()
plt.show()

# MACD

In [ ]:

import yfinance as yf
import plotly
import pandas_ta as pd
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

#MACDs是signal line
#MACD是MACD line = 12EMA - 26EMA
#MACDh是histogram = MACD - signal line, >0 buy, <0 sell

def plotmacd(df):
    # Calculate MACD values using the pandas_ta library
    df.ta.macd(close='close', fast=12, slow=26, signal=9, append=True)  #是否自动找open和close
    # View result
    pd.set_option("display.max_columns", None)  # show all columns
    print(df)


    # Force lowercase (optional)
    df.columns = [x.lower() for x in df.columns]
    # Construct a 2 x 1 Plotly figure


    fig = make_subplots(rows=2, cols=1)
    # price Line
    fig.append_trace(
        go.Scatter(
            x=df['date'],
            y=df['open'],
            line=dict(color='#ff9900', width=1),
            name='open',
            # showlegend=False,
            legendgroup='1',
        ), row=1, col=1
    )

    # Candlestick chart for pricing
    fig.append_trace(
        go.Candlestick(
            x=df['date'],
            open=df['open'],
            high=df['high'],
            low=df['low'],
            close=df['close'],
            increasing_line_color='#ff9900',
            decreasing_line_color='black',
            showlegend=False
        ), row=1, col=1
    )

    # Fast Signal (%k)
    fig.append_trace(
        go.Scatter(
            x=df['date'],
            y=df['macd_12_26_9'],
            line=dict(color='#ff9900', width=2),
            name='macd',
            # showlegend=False,
            legendgroup='2',
        ), row=2, col=1
    )

    # Slow signal (%d)
    fig.append_trace(
        go.Scatter(
            x=df['date'],
            y=df['macds_12_26_9'],
            line=dict(color='#000000', width=2),
            # showlegend=False,
            legendgroup='2',
            name='signal'
        ), row=2, col=1
    )

    # Colorize the histogram values
    colors = np.where(df['macdh_12_26_9'] < 0, '#000', '#ff9900')
    # Plot the histogram
    fig.append_trace(
        go.Bar(
            x=df['date'],
            y=df['macdh_12_26_9'],
            name='histogram',
            marker_color=colors,
        ), row=2, col=1
    )

    # Make it pretty
    layout = go.Layout(
        plot_bgcolor='#efefef',
        # Font Families
        font_family='Monospace',
        font_color='#000000',
        font_size=20,
        xaxis=dict(
            rangeslider=dict(
                visible=False
            )
        )
    )

    # Update options and show plot
    fig.update_layout(layout)
    fig.show()

In [ ]:
#import MACD
import pandas as pd
import re
import datetime
import matplotlib.pyplot as plt
from matplotlib.pyplot import MultipleLocator

if __name__ == '__main__':
    df = pd.read_csv('BTC.csv')        #读数据集
    #start_d = '2018-03-01'
    #end_d = '2018-10-01'
    start_d = datetime.date(2020, 7, 1)   #起始时间
    end_d = datetime.date(2021, 11, 1)    #结束时间
    start_idx = df.loc[df['Date'] == start_d.__format__('%Y-%m-%d')].index[0]
    end_idx = df.loc[df['Date'] == end_d.__format__('%Y-%m-%d')].index[0]
    dfPlot = df.loc[start_idx:end_idx, :]
    c = MACD.plotmacd(dfPlot)  # dfPlot 会增加三列，使用macdh_12_26_9这一列，作为金叉死叉点

NameError: name 'MACD' is not defined

# Crypto-TI

In [ ]:

pip install alpha_vantage pandas ta


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import tensorflow as tf
sns.set() # setting seaborn default for plots
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.feature_selection import SelectKBest, f_classif
from google.colab import files
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from pandas_profiling import ProfileReport
from sklearn.metrics import r2_score, accuracy_score
from sklearn.model_selection import learning_curve
from sklearn.model_selection import train_test_split
from matplotlib import style
import datetime as dt1
from datetime import datetime as dt
import plotly.graph_objs as go
from statsmodels.tsa.seasonal import seasonal_decompose
import keras
import keras.backend as K
import ta
K.clear_session()
key = 'MVPD59K08DSW0ANM'
key = '662ULMA9IRI9706B'
key = 'IEIUKL0X9WEK0E2A'
key = 'BT0IMUWGM1BO0E9P'

In [ ]:
from alpha_vantage.cryptocurrencies import CryptoCurrencies
import matplotlib.pyplot as plt

cc = CryptoCurrencies(key=key, output_format='pandas')
data1, meta_data = cc.get_digital_currency_daily(symbol='BTC', market='USD')
data1 = data1.iloc[::-1]

data1['4b. close (USD)'].plot()
plt.tight_layout()
plt.title('Daily close value for bitcoin (BTC)')
plt.grid()
plt.show()
data1

In [ ]:
from alpha_vantage.cryptocurrencies import CryptoCurrencies
import matplotlib.pyplot as plt

cc = CryptoCurrencies(key=key, output_format='pandas')
data, meta_data = cc.get_digital_currency_daily(symbol='BTC', market='USD')
#data = data.iloc[::-1]

data['4b. close (USD)'].plot()
plt.tight_layout()
plt.title('Daily close value for bitcoin (BTC)')
plt.grid()
plt.show()
data

In [ ]:
import pandas as pd
from ta.utils import dropna
from ta.volatility import BollingerBands


# Load datas


# Initialize Bollinger Bands Indicator
indicator_bb = BollingerBands(close=data1["4b. close (USD)"], window=20, window_dev=2,fillna= True)

# Add Bollinger Bands features
#data['bb_bbm'] = indicator_bb.bollinger_mavg()
#data['bb_bbh'] = indicator_bb.bollinger_hband()
data['bb_bbl'] = indicator_bb.bollinger_lband()

# Add Bollinger Band high indicator
#data['bb_bbhi'] = indicator_bb.bollinger_hband_indicator()

# Add Bollinger Band low indicator
data['bb_bbli'] = indicator_bb.bollinger_lband_indicator()

In [ ]:
import ta
data['sma'] = ta.trend.sma_indicator(data1["4b. close (USD)"], window=14, fillna=True)
data['ema'] = ta.trend.ema_indicator(data1["4b. close (USD)"], window=14, fillna=True)
data['RSI'] = ta.momentum.rsi(data1["4b. close (USD)"], window=14, fillna=True)
data['macd'] = ta.trend.macd_diff(data1["4b. close (USD)"], window_slow=14, window_fast=12, window_sign=9, fillna=True)

data['obv'] = ta.volume.on_balance_volume(data1["4b. close (USD)"], data1["5. volume"], fillna=True)

data['vwap'] = ta.volume.volume_weighted_average_price( data1['2a. high (USD)'], data1['3a. low (USD)'], data1['4b. close (USD)'], data1['5. volume'], window = 14, fillna= True)
data['stoch'] = ta.momentum.stoch(data1['2a. high (USD)'], data1['3a. low (USD)'], data1['4b. close (USD)'], window=14, smooth_window=3, fillna=True) 
data['cci'] = ta.trend.cci(data1['2a. high (USD)'], data1['3a. low (USD)'], data1['4b. close (USD)'], window=20, constant=0.015, fillna=True)
data['adx'] = ta.trend.adx(data1['2a. high (USD)'], data1['3a. low (USD)'], data1['4b. close (USD)'], window=14, fillna=True)



In [ ]:
data['macd']= data['macd'] / (data1["4b. close (USD)"] )
data['sma']= data['sma']/ (data1["4b. close (USD)"] )
data['ema']= data['ema']/ (data1["4b. close (USD)"] )
data['adx']= data['adx']/ (data1["4b. close (USD)"] )
data['vwap']= data['vwap']/ (data1["4b. close (USD)"] )

In [ ]:
data.loc[data['RSI'] < 30, 'RSI'] = -1
data.loc[data['RSI'] > 70, 'RSI'] = 1
data.loc[(data['RSI'] > 30) & (data['RSI'] < 70), 'RSI'] = 0
data['RSI'].plot()

In [ ]:
data['target'] = np.zeros(data1['4b. close (USD)'].size)  
for i in range(data['4a. close (USD)'].size - 7):
    #data['target'][i] = (-data['5. adjusted close'][i] + ((data['5. adjusted close'][i + 7] +  data['5. adjusted close'][i + 6] +  data['5. adjusted close'][i + 5]+  data['5. adjusted close'][i + 4]+  data['5. adjusted close'][i + 3]+  data['5. adjusted close'][i + 2]+  data['5. adjusted close'][i + 1]) / 7)  ) / data['5. adjusted close'][i]
    data['target'][i] = (-data['4b. close (USD)'][i] + data['4b. close (USD)'][i + 7]   ) / data['4b. close (USD)'][i]
    data['target'][i] = data['target'][i] *100
def trend(x):
    if x > 4:        
        return 1
    elif x < -4:        
        return -1
    else:
        return 0
data['target']= data['target'].apply(lambda x:trend(x))    

In [ ]:
data = data.drop(columns=['1a. open (USD)','1b. open (USD)','2a. high (USD)','2b. high (USD)','3a. low (USD)','3b. low (USD)', '4a. close (USD)','6. market cap (USD)'])
data = data.drop(columns=['4b. close (USD)', '5. volume'])


In [ ]:
sns.set(font_scale = 1)
df1 = pd.melt(data, data.columns[-1], data.columns[:-1])


g = sns.FacetGrid( df1, col="variable", hue="target", col_wrap=4,height=2.5,sharex=False,sharey=False)
g.map(sns.kdeplot, "value", shade=True)

plt.show()


In [ ]:
plt.figure(figsize=(12, 12))
heatmap = sns.heatmap(data.corr(), vmin=-1, vmax=1, annot=True, cmap='BrBG')
heatmap.set_title('Correlation Heatmap', fontdict={'fontsize':18}, pad=12);

In [ ]:
target = data['target']
data = data.drop(columns=['adx','obv','target'])
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# transform data
data = pd.DataFrame(scaler.fit_transform(data),columns = data.columns)
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.25)

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
k_fold = KFold(n_splits=5, shuffle=True, random_state=0)

In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(penalty='l2', C=1)
scoring = 'accuracy'
score = cross_val_score(clf, X_train, y_train, cv=k_fold, n_jobs=1, scoring=scoring)
print(score)
print(round(np.mean(score)*100, 2))

In [ ]:
clf.fit(X_train, y_train)
Y_pred = clf.predict(X_test)
print('The accuracy is',accuracy_score(y_test,Y_pred))